# Store CSV to Store Segments

This Jupyter Notebook demonstrates how, by using ArcGIS, you can start with little more than a list of stores with sales volume and coordinates, and end with stores segmented by similar demographic characteristics.

1. Prepare Input Data
2. Create Drive Time Trade Areas
2. Acquire Demographic Analysis Factors Based on Drive Time Trade Areas Around Stores
3. Segment Stores Using KMeans

### Note: Increased IOPub
For visualization, if you did not start this notebook with an increased data rate limit, stop the notebook, go back to the command line, and start Jupyter Notebook using the following command.

`jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000`

## Prepare Data

The data coming in from the CSV file, while the coordinate locations are present in the data, ArcGIS does not yet know how to recognize the data as _spatial_ for subseqnet analysis steps. To accomplish this, we will load the data into a Pandas DataFrame, convert this into an ArcGIS SpatialDataFrame, and finally create an ArcGIS Feature Set, which we will then use for subsequent analysis.

In [1]:
import pandas as pd
import arcgis

Load the data into a Pandas DataFrame from a CSV file.

In [2]:
df = pd.read_csv('./store_locations.csv', index_col='OBJECTID')
df.head(10)

,LOCNUM,SALESVOL,X,Y
OBJECTID,,,,
1,666990510,35495,-121.8430,36.6210
2,653371815,35495,-121.8112,36.6676
3,423468472,35495,-121.9651,36.9753
4,511743478,35495,-121.7740,36.9154
5,404459478,52059,-122.0362,37.3231
6,373128867,84715,-121.9907,37.2928
7,402344537,35495,-122.0323,37.3737
8,637354200,35495,-121.8614,37.2505
9,435039879,35495,-121.8039,37.2499


While the coordinates for each store are contained in an X (longitude) and Y (latitude) field, the data is not yet able to be recognized spatially. We need to create a point geometry for each location in a new field so the data will be recognized as spatial. Once this is done, we also can get rid of the explicity X and Y fields, since the location is now stored in the SHAPE field.

In [3]:
df['SHAPE'] = df.apply(lambda row: arcgis.geometry.Point({'x': row.X, 'y': row.Y, 'spatialReference': {'wkid': 4326}}), axis=1)
df = df.drop(['X', 'Y'], axis=1)
df.head(10)

,LOCNUM,SALESVOL,SHAPE
OBJECTID,,,
1,666990510,35495,"{'x': -121.843, 'spatialReference': {'wkid': 4..."
2,653371815,35495,"{'x': -121.8112, 'spatialReference': {'wkid': ..."
3,423468472,35495,"{'x': -121.9651, 'spatialReference': {'wkid': ..."
4,511743478,35495,"{'x': -121.774, 'spatialReference': {'wkid': 4..."
5,404459478,52059,"{'x': -122.0362, 'spatialReference': {'wkid': ..."
6,373128867,84715,"{'x': -121.9907, 'spatialReference': {'wkid': ..."
7,402344537,35495,"{'x': -122.0323, 'spatialReference': {'wkid': ..."
8,637354200,35495,"{'x': -121.8614, 'spatialReference': {'wkid': ..."
9,435039879,35495,"{'x': -121.8039, 'spatialReference': {'wkid': ..."


Now, with the location data properly formatted to be recoginzed as point geometry, we can create a SpatialDataFrame with the store locations so the data will now be recognized as spatial data.

In [4]:
sdf = arcgis.features.SpatialDataFrame(df)
sdf.set_geometry(col='SHAPE')  # assign the properly formatted shape field to be recognized by the SpatialDataFrame
sdf.reset_index(inplace=True, drop=True)
sdf.head(10)

,LOCNUM,SALESVOL,SHAPE
0,666990510,35495,"{'x': -121.843, 'spatialReference': {'wkid': 4..."
1,653371815,35495,"{'x': -121.8112, 'spatialReference': {'wkid': ..."
2,423468472,35495,"{'x': -121.9651, 'spatialReference': {'wkid': ..."
3,511743478,35495,"{'x': -121.774, 'spatialReference': {'wkid': 4..."
4,404459478,52059,"{'x': -122.0362, 'spatialReference': {'wkid': ..."
5,373128867,84715,"{'x': -121.9907, 'spatialReference': {'wkid': ..."
6,402344537,35495,"{'x': -122.0323, 'spatialReference': {'wkid': ..."
7,637354200,35495,"{'x': -121.8614, 'spatialReference': {'wkid': ..."
8,435039879,35495,"{'x': -121.8039, 'spatialReference': {'wkid': ..."
9,230021602,70990,"{'x': -121.9181, 'spatialReference': {'wkid': ..."


In [5]:
# get a subset to test with, just the first five records
sdf = sdf[:10]
sdf

,LOCNUM,SALESVOL,SHAPE
0,666990510,35495,"{'x': -121.843, 'spatialReference': {'wkid': 4..."
1,653371815,35495,"{'x': -121.8112, 'spatialReference': {'wkid': ..."
2,423468472,35495,"{'x': -121.9651, 'spatialReference': {'wkid': ..."
3,511743478,35495,"{'x': -121.774, 'spatialReference': {'wkid': 4..."
4,404459478,52059,"{'x': -122.0362, 'spatialReference': {'wkid': ..."
5,373128867,84715,"{'x': -121.9907, 'spatialReference': {'wkid': ..."
6,402344537,35495,"{'x': -122.0323, 'spatialReference': {'wkid': ..."
7,637354200,35495,"{'x': -121.8614, 'spatialReference': {'wkid': ..."
8,435039879,35495,"{'x': -121.8039, 'spatialReference': {'wkid': ..."
9,230021602,70990,"{'x': -121.9181, 'spatialReference': {'wkid': ..."


Convert the SpatailDataFrame to a FeatureSet to use as input for subsequent analysis steps.

__NOTE:__ On 23Aug2017, if you are using the developement branch, the `SpatialDataFrame.to_featureset` method will run without errors, but the geometry will be the same for every record. The `SpatialDataFrame.to_featureset` method is simply calling the `FeatureSet.from_dataframe` method, and the culprit likely lies in `arcgis/features/feature.py` at line 526. Until this is rectified, I have created a rough workaround serving. the purposes of this workflow. Calling the method is commented out so it is still there for testing.

In [6]:
# fs_store_locations = sdf.to_featureset()
# fs_store_locations = arcgis.features.FeatureSet.from_dataframe(sdf)
fs_store_locations = arcgis.features.FeatureSet(
    sdf.apply(lambda row: {'attributes': {'LOCNUM': row.LOCNUM}, 'geometry': row.SHAPE}, axis=1).tolist()
)
fs_store_locations

{"fields": ["LOCNUM"], "geometryType": "esriGeometryPoint", "features": [{"geometry": {"x": -121.84299999999992, "spatialReference": {"wkid": 4326}, "y": 36.62100000000007}, "attributes": {"LOCNUM": 666990510}}, {"geometry": {"x": -121.8112, "spatialReference": {"wkid": 4326}, "y": 36.66760000000007}, "attributes": {"LOCNUM": 653371815}}, {"geometry": {"x": -121.96509999999992, "spatialReference": {"wkid": 4326}, "y": 36.9753000000001}, "attributes": {"LOCNUM": 423468472}}, {"geometry": {"x": -121.77399999999993, "spatialReference": {"wkid": 4326}, "y": 36.91540000000003}, "attributes": {"LOCNUM": 511743478}}, {"geometry": {"x": -122.03619999999997, "spatialReference": {"wkid": 4326}, "y": 37.323100000000046}, "attributes": {"LOCNUM": 404459478}}, {"geometry": {"x": -121.99069999999992, "spatialReference": {"wkid": 4326}, "y": 37.292800000000035}, "attributes": {"LOCNUM": 373128867}}, {"geometry": {"x": -122.03229999999998, "spatialReference": {"wkid": 4326}, "y": 37.37370000000007}, "

## Create Drive Time Trade Areas

Intantiate a Web GIS object instance to use for ArcGIS capabilities.

In [7]:
from getpass import getpass

gis_coldbrew = arcgis.gis.GIS(
    url='http://portal.coldbrew.esri.com/portal',
    username='headless', 
    password=getpass('Please enter the headless password: ')
)

Please enter the headless password: ········


Create a service area layer to use for analysis.

In [8]:
service_area_layer = arcgis.network.ServiceAreaLayer(
    url=gis_coldbrew.properties.helperServices.serviceArea.url, 
    gis=gis_coldbrew
)

Get the travel mode, properly formatted, to use for solving.

In [9]:
travel_modes = service_area_layer.retrieve_travel_modes()
travel_mode_drive = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

Create a SpatialDataFrame, and deritive FeatureSet with the columns formatted for input to service area.

__NOTE:__ On 23Aug2017, if you are using the developement branch, the `SpatialDataFrame.to_featureset` method will run without errors, but the geometry will be the same for every record. The `SpatialDataFrame.to_featureset` method is simply calling the `FeatureSet.from_dataframe` method, and the culprit likely lies in `arcgis/features/feature.py` at line 526. Until this is rectified, I have created a rough workaround serving. the purposes of this workflow. Calling the method is commented out so it is still there for testing.

In [10]:
sdf_service_area_input = sdf[['LOCNUM', 'SHAPE']].rename(columns={'LOCNUM': 'Name'})
# fs_service_area_input = sdf_service_area_input.to_featureset()
fs_service_area_input = arcgis.features.FeatureSet(
    sdf_service_area_input.apply(lambda row: {'attributes': {'Name': row.Name}, 'geometry': row.SHAPE}, axis=1).tolist()
)
fs_service_area_input

{"fields": ["Name"], "geometryType": "esriGeometryPoint", "features": [{"geometry": {"x": -121.84299999999992, "spatialReference": {"wkid": 4326}, "y": 36.62100000000007}, "attributes": {"Name": 666990510}}, {"geometry": {"x": -121.8112, "spatialReference": {"wkid": 4326}, "y": 36.66760000000007}, "attributes": {"Name": 653371815}}, {"geometry": {"x": -121.96509999999992, "spatialReference": {"wkid": 4326}, "y": 36.9753000000001}, "attributes": {"Name": 423468472}}, {"geometry": {"x": -121.77399999999993, "spatialReference": {"wkid": 4326}, "y": 36.91540000000003}, "attributes": {"Name": 511743478}}, {"geometry": {"x": -122.03619999999997, "spatialReference": {"wkid": 4326}, "y": 37.323100000000046}, "attributes": {"Name": 404459478}}, {"geometry": {"x": -121.99069999999992, "spatialReference": {"wkid": 4326}, "y": 37.292800000000035}, "attributes": {"Name": 373128867}}, {"geometry": {"x": -122.03229999999998, "spatialReference": {"wkid": 4326}, "y": 37.37370000000007}, "attributes": {

Get the trade areas around the stores.

In [11]:
resp_service_area = service_area_layer.solve_service_area(
    facilities=fs_service_area_input, 
    travel_mode=travel_mode_drive, 
    default_breaks=[8],
    out_sr=4326
)
resp_service_area

{'messages': [],
 'saPolygons': {'features': [{'attributes': {'FacilityID': 10,
     'FromBreak': 0,
     'Name': '230021602 : 0 - 8',
     'ObjectID': 1,
     'Shape_Area': 0.004404040553011247,
     'Shape_Length': 1.7536817767780442,
     'ToBreak': 8},
    'geometry': {'rings': [[[-121.89800071699995, 37.22858238200007],
       [-121.89776229899996, 37.22854042100005],
       [-121.89798736599994, 37.22854042100005],
       [-121.89800071699995, 37.22858238200007]],
      [[-121.96109390299995, 37.28053474400008],
       [-121.961116791, 37.280641556000035],
       [-121.96109390299995, 37.280641556000035],
       [-121.96109390299995, 37.28053474400008]],
      [[-121.961116791, 37.280641556000035],
       [-121.96199226399995, 37.280641556000035],
       [-121.96131706199998, 37.280866623000065],
       [-121.96131706199998, 37.281541824000044],
       [-121.961116791, 37.280641556000035]],
      [[-121.89800071699995, 37.22858238200007],
       [-121.89910888699995, 37.228765488

Convert the FeatureSet to a SpatialDataFrame to make it easier to clean up the data.

In [12]:
sdf_service_area = arcgis.features.FeatureSet(resp_service_area['saPolygons']['features']).df
sdf_service_area

,FacilityID,FromBreak,Name,ObjectID,Shape_Area,Shape_Length,ToBreak,SHAPE
0,10,0,230021602 : 0 - 8,1,0.004404,1.753682,8,"{'rings': [[[-121.89800071699995, 37.228582382..."
1,2,0,653371815 : 0 - 8,2,0.002575,1.174429,8,"{'rings': [[[-121.783676147, 36.64373970000002..."
2,1,0,666990510 : 0 - 8,3,0.002165,1.549343,8,"{'rings': [[[-121.86070632899998, 36.581754684..."
3,3,0,423468472 : 0 - 8,4,0.001892,1.429841,8,"{'rings': [[[-121.90113067599998, 36.974992752..."
4,4,0,511743478 : 0 - 8,5,0.001504,0.952610,8,"{'rings': [[[-121.81197357199994, 36.920644760..."
5,5,0,404459478 : 0 - 8,6,0.003224,2.072019,8,"{'rings': [[[-122.07342719999997, 37.307561874..."
6,6,0,373128867 : 0 - 8,7,0.003855,2.087756,8,"{'rings': [[[-121.94694518999995, 37.292993546..."
7,7,0,402344537 : 0 - 8,8,0.003258,1.426883,8,"{'rings': [[[-122.02689361599994, 37.408876419..."
8,8,0,637354200 : 0 - 8,9,0.003071,1.669440,8,"{'rings': [[[-121.82679557799997, 37.237249374..."
9,9,0,435039879 : 0 - 8,10,0.003558,2.132018,8,"{'rings': [[[-121.77222251899997, 37.273904800..."


Get the LOCNUM out of the Name field, and drop all the fields besides LOCNUM and SHAPE.

In [13]:
sdf_service_area['LOCNUM'] = sdf_service_area['Name'].apply(lambda name: name.split(':')[0])
sdf_service_area = sdf_service_area.drop(['FacilityID', 'FromBreak', 'ObjectID', 'ToBreak', 
                                        'Name', 'Shape_Area', 'Shape_Length'], axis=1)
sdf_service_area

,SHAPE,LOCNUM
0,"{'rings': [[[-121.89800071699995, 37.228582382...",230021602
1,"{'rings': [[[-121.783676147, 36.64373970000002...",653371815
2,"{'rings': [[[-121.86070632899998, 36.581754684...",666990510
3,"{'rings': [[[-121.90113067599998, 36.974992752...",423468472
4,"{'rings': [[[-121.81197357199994, 36.920644760...",511743478
5,"{'rings': [[[-122.07342719999997, 37.307561874...",404459478
6,"{'rings': [[[-121.94694518999995, 37.292993546...",373128867
7,"{'rings': [[[-122.02689361599994, 37.408876419...",402344537
8,"{'rings': [[[-121.82679557799997, 37.237249374...",637354200
9,"{'rings': [[[-121.77222251899997, 37.273904800...",435039879


Convert the service area SpatialDataFrame to a FeatureSet for geoenrichment.

__NOTE:__ On 23Aug2017, if you are using the developement branch, the `SpatialDataFrame.to_featureset` method will run without errors, but the geometry will be the same for every record. The `SpatialDataFrame.to_featureset` method is simply calling the `FeatureSet.from_dataframe` method, and the culprit likely lies in `arcgis/features/feature.py` at line 526. Until this is rectified, I have created a rough workaround serving. the purposes of this workflow. Calling the method is commented out so it is still there for testing.

In [14]:
# fs_service_area = sdf_service_area.to_featureset()
fs_service_area = arcgis.features.FeatureSet(
    sdf_service_area.apply(lambda row: {'attributes': {'LOCNUM': row.LOCNUM}, 'geometry': row.SHAPE}, axis=1).tolist()
)

## Perform Geoenrichment

This is some set-up for stuff not currently working in our instance.

In [15]:
trade_area_drive_time = 8  # in minutes
study_area_options = '{"areaType":"DriveTimeBuffer","bufferUnits":"esriDriveTimeUnitsMinutes",' + \
        '"bufferRadii":' + '[{drive_time}]'.format(drive_time=trade_area_drive_time) + '}"'
study_area_options = '{"areaType":"DriveTimeBuffer","bufferUnits":"esriDriveTimeUnitsMinutes","bufferRadii":[5]}'

Since the ArcGIS Python API requires a published layer to use the built in Geoenrichment method, but we just want to use a FeatureSet as input, we utilize the ArcGIS Python API's built in `post` method, which takes care of the token authetication, and also has the `urllib.encode` method built in for converting the payload from a dictionary for the post call

In [16]:
url_geoenrich = gis_coldbrew.properties.helperServices.geoenrichment.url + "/Geoenrichment/Enrich"
payload = {
    'studyAreas': fs_service_area.features,
#    'analysisVariables': enrichment_variables,
    'dataCollections': '["KeyUSFacts"]',
#    'studyAreasOptions': study_area_options,  # this can be used if AGOL or a correctly configured BA Server is used
    'f': 'json'
}
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
}
resp_enrich = gis_coldbrew._con.post(url_geoenrich, postdata=payload)
resp_enrich

{'messages': [],
 'results': [{'dataType': 'GeoEnrichmentResult',
   'paramName': 'GeoEnrichmentResult',
   'value': {'FeatureSet': [{'displayFieldName': '',
      'features': [{'attributes': {'AREA_ID': '0',
         'AVGHHSZ_CY': 2.7,
         'AVGHINC_CY': 125882,
         'AVGHINC_FY': 137060,
         'AVGVAL_CY': 738732,
         'AVGVAL_FY': 763828,
         'DIVINDX_CY': 67.7,
         'FAMGRW10CY': 0.59,
         'FAMGRWCYFY': 1.02,
         'GQPOP_CY': 1011,
         'HHGRW10CY': 0.56,
         'HHGRWCYFY': 1,
         'HasData': 1,
         'ID': '0',
         'LOCNUM': '230021602 ',
         'MEDHINC_CY': 99564,
         'MEDHINC_FY': 108113,
         'MEDVAL_CY': 709202,
         'MEDVAL_FY': 731353,
         'MHIGRWCYFY': 1.66,
         'OBJECTID': 1,
         'OWNER_CY': 28494,
         'OWNER_FY': 29769,
         'PCIGRWCYFY': 1.61,
         'PCI_CY': 46598,
         'PCI_FY': 50466,
         'POPGRW10CY': 0.77,
         'POPGRWCYFY': 1.09,
         'RENTER_CY': 15231,


From the response, we can pull out the data, and convert it into a FeatureSet.

In [17]:
fs_enrich = arcgis.features.FeatureSet(
    features=resp_enrich['results'][0]['value']['FeatureSet'][0]['features'], 
    fields=resp_enrich['results'][0]['value']['FeatureSet'][0]['fields']
)
fs_enrich

{"fields": [{"type": "esriFieldTypeOID", "name": "OBJECTID", "alias": "Object ID"}, {"type": "esriFieldTypeString", "name": "ID", "alias": "ID", "length": 256}, {"type": "esriFieldTypeString", "name": "LOCNUM", "alias": "LOCNUM", "length": 256}, {"type": "esriFieldTypeString", "name": "sourceCountry", "alias": "sourceCountry", "length": 256}, {"type": "esriFieldTypeString", "name": "AREA_ID", "alias": "AREA_ID", "length": 256}, {"type": "esriFieldTypeInteger", "name": "HasData", "alias": "HasData"}, {"type": "esriFieldTypeString", "name": "aggregationMethod", "alias": "aggregationMethod", "length": 256}, {"component": "demographics", "type": "esriFieldTypeDouble", "name": "AVGHHSZ_CY", "decimals": 2, "alias": "2016 Average Household Size", "fullName": "KeyUSFacts.AVGHHSZ_CY", "units": "count"}, {"component": "demographics", "type": "esriFieldTypeDouble", "name": "AVGHINC_CY", "currency": "$", "decimals": 0, "alias": "2016 Average Household Income", "fullName": "KeyUSFacts.AVGHINC_CY", 

Now, we can convert the FeatureSet to a SpatialDataFrame to be ready to do some hard core number crunching.

In [18]:
sdf_enrich = fs_enrich.df
sdf_enrich

,AREA_ID,AVGHHSZ_CY,AVGHINC_CY,AVGHINC_FY,AVGVAL_CY,AVGVAL_FY,DIVINDX_CY,FAMGRW10CY,FAMGRWCYFY,GQPOP_CY,...,TOTHU_CY,TOTHU_FY,TOTPOP00,TOTPOP10,TOTPOP_CY,TOTPOP_FY,VACANT_CY,VACANT_FY,aggregationMethod,sourceCountry
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,0,2.70,125882,137060,738732,763828,67.7,0.59,1.02,1011,...,45112,47491,110878,113636,119219,125848,1388,1531,BlockApportionment:US.BlockGroups,US
2,1,2.81,73625,78876,466059,524706,85.0,0.37,0.61,1754,...,8341,8598,27161,22797,23578,24435,570,568,BlockApportionment:US.BlockGroups,US
3,2,2.96,69864,76498,521829,581500,87.8,0.46,0.65,226,...,13810,14236,41925,36591,38170,39622,984,1003,BlockApportionment:US.BlockGroups,US
4,3,2.39,86793,95374,602246,660258,64.7,0.34,0.75,840,...,21225,21918,45236,44523,46061,47931,2292,2276,BlockApportionment:US.BlockGroups,US
5,4,3.92,59183,63503,433158,497347,87.7,0.54,0.93,571,...,11699,12239,39191,42121,44207,46385,563,576,BlockApportionment:US.BlockGroups,US
6,5,2.89,160793,173699,1029682,1050982,54.3,0.74,1.12,529,...,32037,33915,79101,85079,90446,96068,916,1008,BlockApportionment:US.BlockGroups,US
7,6,2.82,134493,145621,902124,920311,74.6,0.63,1.06,633,...,41615,43924,107284,108842,114273,120768,1271,1408,BlockApportionment:US.BlockGroups,US
8,7,2.56,130244,141229,757154,785869,75.7,0.92,1.34,494,...,42063,44977,88880,96958,104290,111940,1495,1627,BlockApportionment:US.BlockGroups,US
9,8,2.87,117242,128162,660828,686797,76.2,0.52,0.96,348,...,30368,31890,79360,81330,85062,89599,831,923,BlockApportionment:US.BlockGroups,US


## Segment Stores using KMeans Clustering